In [1]:
import scraping
import pickle
import pandas as pd
import tqdm

import os
import sys
sys.path.append(os.pardir)
import common.com as common
from common import const,utils
import traceback
import time

# データ取得・保存処理
- netkeiba.comから指定された期間に開催された全レース結果を取得し、origin_race_dataテーブルに保存する

In [2]:
# 開催日一覧を取得
from_date = "2023-12"
to_date = "2024-12"
kaisai_date_list = scraping.scrape_kaisai_date(from_=from_date, to_=to_date)

  0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
# 開催日に開かれたレースのレースID一覧を取得
race_id_list = scraping.scrape_race_id_list(kaisai_date_list)

  0%|          | 0/115 [00:00<?, ?it/s]

In [4]:
# レースID一覧を保存
race_id_list_file_path=f"{const.RACE_ID_LIST_DIR}{from_date}_{to_date}.pickle"
with open(race_id_list_file_path, "wb") as f:
    pickle.dump(race_id_list, f)

In [5]:
# レースID一覧を読み込み
with open(race_id_list_file_path, 'rb') as file:
    race_id_list = pickle.load(file)

# レースID一覧からレースデータを取得
race_datas=[]
for race_id in tqdm.tqdm(race_id_list):
    try:
        race_data = scraping.scrape_race_result(race_id)
        race_datas.extend(race_data)
    except Exception as e:
        print(f"ERROR: in race_id:{race_id}")
        print(traceback.format_exc())    # いつものTracebackが表示される
        traceback.print_exc()                 # これでも同じ
        continue
    time.sleep(2)

  0%|          | 0/3742 [00:00<?, ?it/s]

100%|██████████| 3742/3742 [25:12<00:00,  2.47it/s]


In [6]:
# レースデータをcsvに保存
df = pd.DataFrame(race_datas)

df.columns = const.ORIGIN_RACE_DATA_HEADERS
csv_file_path=race_id_list_file_path.replace('.pickle', '.csv').replace('race_id_lists', 'race_datas')
df.to_csv(csv_file_path, index=False)

In [7]:
# postgresにデータを保存
common.insert_data(csv_file_path,table_name='origin_race_data')